### Agent-Lab: Multi-Agent -> Coder

Objective of this notebook is evaluating and adapting a [Multi-Agent Supervisor Architecture](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#supervisor) with coordinator and execution planning steps.

---


In [1]:
%%capture
import json
import os
import nest_asyncio
from IPython.display import Markdown, display
from dotenv import load_dotenv
from notebooks import experiment_utils
from app.core.container import Container
from app.interface.api.messages.schema import MessageRequest

os.chdir("..")
load_dotenv()
nest_asyncio.apply()

# start dependency injection container
container = Container()
container.init_resources()
container.wire(modules=[__name__])

# get checkpointer instance
graph_persistence_factory = container.graph_persistence_factory()
checkpointer = graph_persistence_factory.build_checkpoint_saver()

---
### XAI Grok Coder

In [2]:
# Create Workflow
xai_agent = experiment_utils.create_xai_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="grok-code-fast", api_key=os.getenv("XAI_API_KEY")
)
xai_coder_agent = container.coordinator_planner_supervisor_agent()
xai_workflow_builder = xai_coder_agent.get_workflow_builder(xai_agent["id"])
xai_workflow = xai_workflow_builder.compile(checkpointer=checkpointer)

In [3]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="With coder, generate a simple hello world with FastAPI in Python. No need to setup libraries, just generate the code as a text report that makes use of markdown formatting for reading. Make sure to include code examples in the final report.",
    agent_id=xai_agent["id"],
)

inputs = xai_coder_agent.get_input_params(message, schema="public")
config = xai_coder_agent.get_config(xai_agent["id"])
result = xai_workflow.invoke(inputs, config)
ai_message_content, workflow_state = xai_coder_agent.format_response(result)

In [4]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "The user wants a simple Hello World example using FastAPI in Python, generated as a text report with markdown formatting, including code examples, without needing to set up libraries.",
  "title": "Generate a Simple Hello World FastAPI Example in Python",
  "steps": [
    {
      "agent_name": "coder",
      "title": "Generate FastAPI Hello World Code",
      "description": "Use coder to create a simple Hello World application code using FastAPI. Output the code in a markdown-formatted report, including code examples and explanations."
    },
    {
      "agent_name": "reporter",
      "title": "Compile Final Report",
      "description": "Use reporter to compile the generated code into a professional final report, ensuring markdown formatting and inclusion of code examples."
    }
  ]
}
```

In [5]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**

# Simple Hello World Example with FastAPI in Python

## Overview
This report provides a simple "Hello World" example using FastAPI, a modern, fast web framework for building APIs with Python. The example demonstrates how to create a basic FastAPI application that responds to HTTP requests with a "Hello, World!" message. No library setup is required, as FastAPI is assumed to be installed (e.g., via `pip install fastapi uvicorn` in a real environment).

## Requirements
- Python 3.7+
- FastAPI library
- Uvicorn (for running the server)

## Implementation Steps
1. **Install Dependencies** (if needed in a real setup): Run `pip install fastapi uvicorn`.
2. **Create the Application**: Write a Python script with FastAPI to define routes.
3. **Run the Server**: Use Uvicorn to start the development server.
4. **Test the Endpoint**: Access the API endpoint in a browser or via tools like curl.

## Code Example
Below is the complete Python code for the Hello World application. Save this as `main.py` and run it with `uvicorn main:app --reload` to start the server on `http://127.0.0.1:8000`.

```python
from fastapi import FastAPI

# Create a FastAPI instance
app = FastAPI()

# Define a simple route that returns "Hello, World!"
@app.get("/")
async def read_root():
    return {"message": "Hello, World!"}
```

### Explanation of the Code
- `from fastapi import FastAPI`: Imports the FastAPI class.
- `app = FastAPI()`: Initializes the FastAPI application.
- `@app.get("/")`: Decorator that defines a GET endpoint at the root URL ("/").
- `async def read_root()`: Asynchronous function that handles the request and returns a JSON response with the message.

## Running and Testing
1. Save the code to a file named `main.py`.
2. In your terminal, run: `uvicorn main:app --reload`
3. Open a browser and navigate to `http://127.0.0.1:8000` to see the JSON response: `{"message": "Hello, World!"}`.
4. Alternatively, use curl: `curl http://127.0.0.1:8000` to get the same output.

## Additional Notes
- This is a minimal example. FastAPI supports more advanced features like request validation, dependency injection, and automatic API documentation (available at `http://127.0.0.1:8000/docs` when running).
- For production, consider using a proper ASGI server like Uvicorn or Gunicorn.
- Error handling and more routes can be added as needed.

This example is efficient, follows best practices for FastAPI, and handles the basic requirement of a Hello World response. If you need expansions (e.g., adding more endpoints), let me know!

---
### Anthropic Coder

In [6]:
# Create Workflow
anthropic_agent = experiment_utils.create_anthropic_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="claude-3-5-haiku-latest", api_key=os.getenv("ANTHROPIC_API_KEY")
)
anthropic_coder_agent = container.coordinator_planner_supervisor_agent()
anthropic_workflow_builder = anthropic_coder_agent.get_workflow_builder(anthropic_agent["id"])
anthropic_workflow = anthropic_workflow_builder.compile(checkpointer=checkpointer)

In [7]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="With coder, generate a simple hello world with FastAPI in Python. No need to setup libraries, just generate the code as a text report that makes use of markdown formatting for reading. Make sure to include code examples in the final report.",
    agent_id=anthropic_agent["id"],
)

inputs = anthropic_coder_agent.get_input_params(message, schema="public")
config = anthropic_coder_agent.get_config(anthropic_agent["id"])
result = anthropic_workflow.invoke(inputs, config)
ai_message_content, workflow_state = anthropic_coder_agent.format_response(result)

In [8]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "I'll use the coder agent to generate a simple FastAPI Hello World application and create a markdown-formatted code report.",
  "title": "Generate FastAPI Hello World Python Code",
  "steps": [
    {
      "agent_name": "coder",
      "title": "Create FastAPI Hello World Code",
      "description": "Write a simple FastAPI application that returns a hello world message. The code will demonstrate basic FastAPI routing and response handling."
    },
    {
      "agent_name": "reporter",
      "title": "Generate Markdown Report",
      "description": "Create a professional report with the generated FastAPI code, including explanations and code blocks."
    }
  ]
}
```

In [9]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**



📄 **FastAPI Hello World: A Concise Guide**

## Overview
This guide demonstrates a simple FastAPI application that returns a "Hello, World!" message, showcasing the basic structure and routing capabilities of FastAPI.

### Prerequisites
- Python 3.7+
- FastAPI
- Uvicorn (ASGI server)

### Code Example

```python
from fastapi import FastAPI

# Create a FastAPI application instance
app = FastAPI()

# Define a root endpoint that returns a hello world message
@app.get("/")
async def read_root():
    """
    Root endpoint that returns a hello world JSON response.
    
    Returns:
        dict: A greeting message
    """
    return {"message": "Hello, World!"}

# Optional: Additional endpoint demonstrating parameter handling
@app.get("/hello/{name}")
async def hello_name(name: str):
    """
    Personalized greeting endpoint.
    
    Args:
        name (str): Name to include in the greeting
    
    Returns:
        dict: A personalized greeting message
    """
    return {"message": f"Hello, {name}!"}
```

## How to Run
To run this FastAPI application, you would typically use Uvicorn:
```bash
uvicorn main:app --reload
```

## Key Components
- `FastAPI()`: Creates the main application instance
- `@app.get()`: Decorator to define route handlers
- `async def`: Defines asynchronous route handler functions
- Returns JSON responses by default

## Features Demonstrated
✅ Basic routing
✅ Async endpoint handlers
✅ Simple JSON responses
✅ Path parameter handling

## Recommendations
- For production, use proper ASGI servers like Uvicorn or Hypercorn
- Add error handling and input validation as needed
- Consider using type hints for better code quality

By following this example, you can quickly set up a basic FastAPI application with routing and response handling. The code provides a clean, modern approach to building web APIs in Python.

---
### OpenAI Coder

In [10]:
# Create Workflow
openai_agent = experiment_utils.create_openai_agent(
    agent_type="coordinator_planner_supervisor", llm_tag="gpt-5-nano", api_key=os.getenv("OPENAI_API_KEY")
)
openai_coder_agent = container.coordinator_planner_supervisor_agent()
openai_workflow_builder = openai_coder_agent.get_workflow_builder(openai_agent["id"])
openai_workflow = openai_workflow_builder.compile(checkpointer=checkpointer)

In [11]:
%%capture

message = MessageRequest(
    message_role="human",
    message_content="With coder, generate a simple hello world with FastAPI in Python. No need to setup libraries, just generate the code as a text report that makes use of markdown formatting for reading.",
    agent_id=openai_agent["id"],
)

inputs = openai_coder_agent.get_input_params(message, schema="public")
config = openai_coder_agent.get_config(openai_agent["id"])
result = openai_workflow.invoke(inputs, config)
ai_message_content, workflow_state = openai_coder_agent.format_response(result)

In [12]:
display(
    Markdown(
        f"**Execution Plan:**\n```json\n{json.dumps(workflow_state.get("execution_plan"), indent=2)}\n```"
    )
)

**Execution Plan:**
```json
{
  "thought": "You're asking for a simple Python FastAPI 'Hello World' example. I'll provide a minimal FastAPI app as a Markdown-formatted code block. No library installation steps, only the code and a tiny run instruction.",
  "title": "Generate a minimal FastAPI Hello World app in Python (Markdown-formatted code report)",
  "steps": [
    {
      "agent_name": "coder",
      "title": "Create minimal FastAPI Hello World app code",
      "description": "Produce a concise Python script that implements a FastAPI app with a root endpoint returning a Hello, World! message. Output should be a Markdown-formatted code block. Include a standard guard to run the app with uvicorn when executed as the main module."
    },
    {
      "agent_name": "reporter",
      "title": "Present final report in Markdown",
      "description": "Provide a polished Markdown report that includes a brief title, a short description, and the code block from Step 1. Include a short instruction line on how to run the app (e.g., 'uvicorn main:app --reload')."
    }
  ]
}
```

In [13]:
display(Markdown(f"**AI Message Content:**\n\n{ai_message_content}"))

**AI Message Content:**

Executive Summary
- This report provides a minimal FastAPI Python application that serves a single endpoint at the root (/) and returns a JSON message "Hello, World!". The code is presented as a Markdown-formatted block, suitable for quick reading and integration into demonstrations or tutorials.

Key Findings
- Endpoint: The application defines a single GET endpoint at the root path ("/").
- Response: The endpoint returns a JSON object with the key "message" and value "Hello, World!".
- Execution: The application is designed to run with uvicorn using a standard Python module entry point (if __name__ == "__main__").
- Dependencies (implied): FastAPI and uvicorn are used to run the application. No installation steps are included in the report, per request.

Detailed Analysis
- Code Structure:
  - The script creates a FastAPI instance named app.
  - A single asynchronous function read_root is decorated with @app.get("/") to handle GET requests at the root path.
  - The function returns a Python dict which FastAPI serializes to JSON as {"message": "Hello, World!"}.
  - The if __name__ == "__main__" block imports uvicorn and runs the app, exposing it on host 0.0.0.0 and port 8000.
- Behavior:
  - Accessing the root URL via HTTP GET yields the JSON response {"message": "Hello, World!"}.
  - The app is suitable for quick demonstrations or as a template for expanding to additional routes.

Code
```python
# main.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def read_root():
    return {"message": "Hello, World!"}

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
```

Run Instructions
- Command to run: uvicorn main:app --reload --port 8000
- Access: http://localhost:8000/ (or http://0.0.0.0:8000/ when running on a server)

Conclusions and Recommendations
- This is a compact, functional Hello World example using FastAPI suitable for initial demonstrations or scaffolding for further development.
- If expanding functionality, consider adding more endpoints, input validation, and error handling, and documenting with FastAPI’s interactive docs (auto-generated).
- For production deployments, assess security, logging, and deployment configuration beyond this minimal example.